In [71]:
import codecs
import pandas as pd
df_vnexpress = pd.read_json(codecs.open('data/vnexpress.json','r','utf-8'))
df_dantri = pd.read_json(codecs.open('data/dantri.json','r','utf-8'))
df_vietnamnet = pd.read_json(codecs.open('data/vietnamnet.json','r','utf-8'))

data_vnexpress = pd.DataFrame(df_vnexpress.response['docs'])[['url','content']]
data_dantri = pd.DataFrame(df_dantri.response['docs'])[['url','content']]
data_vietnamnet = pd.DataFrame(df_vietnamnet.response['docs'])[['url','content']]

In [72]:
print(data_dantri.shape[0])
print(data_vnexpress.shape[0])
print(data_vietnamnet.shape[0])

19682
15539
2641


In [73]:
# Get all urls in domain https://dantri.com.vn
data_dantri = data_dantri[data_dantri.url.str.match(pat='^https://dantri\.com\.vn')]
# Get all urls in domain https://vnexpress.vn
data_vnexpress = data_vnexpress[data_vnexpress.url.str.match(pat='^https://vnexpress\.net')]

In [74]:
data_dantri.to_csv('dantri.csv')
data_vnexpress.to_csv('vnexpress.csv')

In [75]:
data_vnexpress.shape

(15518, 2)

In [76]:
regex_list_remove_url_vnexpress= [
    '^.*trac-nghiem.*',
    '^.*\/index.html$',
    '^.*topic-[0-9]*\.html$',
    '^.*/vnexpress\.net\/[a-zA-Z0-9\-]*\.html',
    '^.*/vnexpress\.net\/cuoi\/',
    '^.*/vnexpress\.net\/tac-gia\/',
    '^.*/vnexpress\.net\/hoi-nghi-thuong-dinh-my-trieu\/',
    '^.*/vnexpress\.net\/infographic\/',
    '^.*/vnexpress\.net\/infographics\/',
    '^.*/vnexpress\.net\/interactive\/',
    '^.*/vnexpress\.net\/longform\/',
    '^.*/vnexpress\.net\/phong-van-truc-tuyen\/',
    '^.*/vnexpress\.net\/wef-asean\/',
    '^.*/vnexpress\.net\/goc-nhin\/',
    '^.*/vnexpress\.net\/apec-2017\/',
    '^.*/vnexpress\.net\/tu-van-online\/',
    '^.*/vnexpress\.net\/tam-su\/',
    '^.*/vnexpress\.net\/y-kien\/',
    '^.*/video\/'
]

for regex in regex_list_remove_url_vnexpress:
    print( '%.5d:  ' % data_vnexpress[data_vnexpress.url.str.match(regex)].shape[0] +regex)
    data_vnexpress = data_vnexpress[~data_vnexpress.url.str.match(regex)] 

00040:  ^.*trac-nghiem.*
00029:  ^.*\/index.html$
00013:  ^.*topic-[0-9]*\.html$
00001:  ^.*/vnexpress\.net\/[a-zA-Z0-9\-]*\.html
00489:  ^.*/vnexpress\.net\/cuoi\/
00047:  ^.*/vnexpress\.net\/tac-gia\/
00223:  ^.*/vnexpress\.net\/hoi-nghi-thuong-dinh-my-trieu\/
00001:  ^.*/vnexpress\.net\/infographic\/
00242:  ^.*/vnexpress\.net\/infographics\/
00055:  ^.*/vnexpress\.net\/interactive\/
00125:  ^.*/vnexpress\.net\/longform\/
00013:  ^.*/vnexpress\.net\/phong-van-truc-tuyen\/
00001:  ^.*/vnexpress\.net\/wef-asean\/
00213:  ^.*/vnexpress\.net\/goc-nhin\/
00006:  ^.*/vnexpress\.net\/apec-2017\/
00064:  ^.*/vnexpress\.net\/tu-van-online\/
00219:  ^.*/vnexpress\.net\/tam-su\/
00254:  ^.*/vnexpress\.net\/y-kien\/
00008:  ^.*/video\/


In [77]:
data_vnexpress.shape

(13475, 2)

In [78]:
print(data_vnexpress[data_vnexpress.url.str.match('https://vnexpress.net/bong-da')].shape[0])
print(data_vnexpress[data_vnexpress.url.str.match('https://vnexpress.net/doi-song')].shape[0])
print(data_vnexpress[data_vnexpress.url.str.match('https://vnexpress.net/du-lich')].shape[0])
print(data_vnexpress[data_vnexpress.url.str.match('https://vnexpress.net/giai-tri')].shape[0])
print(data_vnexpress[data_vnexpress.url.str.match('https://vnexpress.net/giao-duc')].shape[0])
print(data_vnexpress[data_vnexpress.url.str.match('https://vnexpress.net/khoa-hoc')].shape[0])
print(data_vnexpress[data_vnexpress.url.str.match('https://vnexpress.net/kinh-doanh')].shape[0])
print(data_vnexpress[data_vnexpress.url.str.match('https://vnexpress.net/oto-xe-may')].shape[0])
print(data_vnexpress[data_vnexpress.url.str.match('https://vnexpress.net/phap-luat')].shape[0])
print(data_vnexpress[data_vnexpress.url.str.match('https://vnexpress.net/so-hoa')].shape[0])
print(data_vnexpress[data_vnexpress.url.str.match('https://vnexpress.net/suc-khoe')].shape[0])
print(data_vnexpress[data_vnexpress.url.str.match('https://vnexpress.net/the-gioi')].shape[0])
print(data_vnexpress[data_vnexpress.url.str.match('https://vnexpress.net/the-thao')].shape[0])
print(data_vnexpress[data_vnexpress.url.str.match('https://vnexpress.net/thoi-su')].shape[0])



926
517
936
1502
999
486
1551
305
1221
1158
904
1576
636
757


In [79]:
regex_list_remove_url_dantri = [
    '^.*dantri.com.vn\/[a-zA-Z0-9\-]*\.htm',
    '^.*\/trang-[0-9]*.htm',
    '^.*\/event\/',
    '^.*tour-hay-khuyen-mai'
]

for regex in regex_list_remove_url_dantri:
    print( '%5d:  ' % data_dantri[data_dantri.url.str.match(regex)].shape[0] +regex)
#     data_vnexpress[data_vnexpress.url.str.match(regex)] 

   27:  ^.*dantri.com.vn\/[a-zA-Z0-9\-]*\.htm
  186:  ^.*\/trang-[0-9]*.htm
  118:  ^.*\/event\/
    6:  ^.*tour-hay-khuyen-mai


In [80]:
data_dantri = data_dantri[~data_dantri.url.str.match('^.*dantri.com.vn\/(video|\
co-vua|\
ban-doc|\
bao-ve-gia-dinh-viet|\
blog|\
cac-ngoi-truong-danh-tieng|\
cam-nang-hoi-dap|\
chuyen-la|\
co-hoi-du-hoc|\
cong-dong|\
dien-dan|\
doi-song-van-hoa|\
du-hoc)\/')]

In [81]:
data_dantri = data_dantri[~data_dantri.url.str.match('^.*dantri.com.vn\/video\/')]

In [82]:
data_dantri[data_dantri.url.str.match('^.*dantri.com.vn\/(video|\
co-vua|\
ban-doc|\
bao-ve-gia-dinh-viet|\
blog|\
cac-ngoi-truong-danh-tieng|\
cam-nang-hoi-dap|\
chuyen-la|\
co-hoi-du-hoc|\
cong-dong|\
dien-dan|\
doi-song-van-hoa|\
du-hoc|\
the-gioi)\/')].shape

(3342, 2)

In [83]:
data_dantri[data_dantri.url.str.match('^.*dantri.com.vn\/(\
doanh-nghiep|\
doi-song|\
ho-so|\
hoi-am|\
khoi-nghiep|\
kieu-bao|\
moi-truong|\
mon-ngon-diem-dep|\
multimedia|\
nghe-nghiep|\
nguoi-viet-tre|\
nha-dat|\
nha-dep|\
nhan-tai-dat-viet|\
nhip-song-tre|\
sang-tao-viet|\
sao-viet|\
tai-chinh-dau-tu|\
tam-long-nhan-ai|\
the-gioi|\
thi-truong-xe|\
thi-truong|\
thoi-trang|\
tieng-anh-du-hoc|\
tin-tuyen-sinh|\
tinh-yeu-gioi-tinh|\
tu-van|\
tuyen-sinh|\
ung-ho|\
van-hoa|\
viec-lam|\
vong-quay-du-lich|\
world-cup-2018|\
)\/')].shape

(6838, 2)

In [84]:
data_dantri.shape

(18242, 2)

In [85]:
data_dantri.to_csv('dantri.csv')
data_vnexpress.to_csv('vnexpress.csv')

data_vnexpress = data_vnexpress['url','content'].str.replace('^(.*[0-9]{2}\:[0-9]{2})',repl='')



KeyError: ('url', 'content')

In [69]:
data_vnexpress

0         CD Leganés   Ngoại hạng Anh Hoeness: 'Khi Pep...
1         (GMT+7) Hồ Quang Hiếu: 'Tôi làm lại cuộc đời ...
2         (GMT+7) Airbus thắng đơn hàng 35 tỷ USD khi B...
3         (GMT+7) Ba phụ nữ đánh ghen bằng nước mắm và ...
4         (GMT+7) Nhiều người liên quan 'Vũ Nhôm' đang ...
5         (GMT+7) Ông Trương Duy Nhất liên quan đến Vũ ...
6         (GMT+7) Apple ra dịch vụ truyền hình riêng, t...
7         (GMT+7) Apple tiết lộ sân khấu trước sự kiện ...
8         (GMT+7) Nỗi buồn sau chiến thắng của HLV Park...
9         (GMT+7) Bác sĩ xin lỗi vì khuyên bệnh nhân đế...
10        (GMT+7) Nam thanh niên đi lạc 3 ngày trên núi...
11        (GMT+7) Người phụ nữ tuyên truyền ‘vong báo o...
12        (GMT+7) Quảng Ninh họp báo vụ chùa Ba Vàng Ch...
13        (GMT+7) Vi phạm của tài xế sẽ được cập nhật t...
14        CD Leganés   Ngoại hạng Anh Hoeness: 'Khi Pep...
15        (GMT+7) Bài toán mã vạch của kỳ thi IMAS 2016...
16        (GMT+7) Vì sao tiến sĩ khó thành tỷ phú? Nhiề.

In [57]:
data_vnexpress.content.str.match('^(.*[0-9]{2}\:[0-9]{2} \(GMT\+7\))').shape

(13475,)

In [58]:
data_vnexpress.shape

(13475, 2)

In [36]:
data_dantri.url.to_csv('url.csv',header=False,index=False)

In [37]:
data_vnexpress.url.to_csv('url2.csv',header=False,index=False)

In [38]:
data_vietnamnet.url.to_csv('url3.csv',header=False,index=False)